In [38]:
%pip uninstall -qqqy google-generativeai google-ai-generativelanguage
%pip install -qU "langchain[google-genai]" langchain langgraph>0.2.27 langchain_community pypdf faiss-cpu langchain-tavily langchain-huggingface langgraph-checkpoint-sqlite
import os
import getpass
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
load_dotenv()
# .env should also have TAVILY_API_KEY
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "default"
if not os.environ.get("TAVILY_API_KEY"):
  os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter API key for Tavily: ")
if not os.environ.get("HF_TOKEN"):
  os.environ["HF_TOKEN"] = getpass.getpass("Enter API key for Hugging Face: ")
if not os.environ.get("GEMINI_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")
else:
  os.environ["GOOGLE_API_KEY"] = os.environ.get("GEMINI_API_KEY")
model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [23]:
from langchain_tavily import TavilySearch
search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco, CA', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1756599245, 'localtime': '2025-08-30 17:14'}, 'current': {'last_updated_epoch': 1756598400, 'last_updated': '2025-08-30 17:00', 'temp_c': 21.2, 'temp_f': 70.2, 'is_day': 1, 'condition': {'text': 'Partly Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 10.1, 'wind_kph': 16.2, 'wind_degree': 270, 'wind_dir': 'W', 'pressure_mb': 1014.0, 'pressure_in': 29.94, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 69, 'cloud': 0, 'feelslike_c': 21.2, 'feelslike_f': 70.2, 'windchill_c': 18.0, 'windchill_f': 64.4, 'heatindex_c': 18.0, 'heatindex_f': 64.5, 'dewpoint_c': 14.9,

In [26]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")
model_with_tools = model.bind_tools(tools)
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])
print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hi, how can I help you today?

Tool calls: []


In [27]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}
# First turn
resp1 = agent_executor.invoke({"messages": [
    {"role": "user", "content": "Hi!, Search for the weather in Mumbai as of now"}
]}, config)

# Second turn — same thread_id, memory kicks in
resp2 = agent_executor.invoke({"messages": [
    {"role": "user", "content": "Now tell me if I need an raincoat"}
]}, config)

# Third turn — still same thread_id
resp3 = agent_executor.invoke({"messages": [
    {"role": "user", "content": "Remind me what I asked first"}
]}, config)
for m in resp3["messages"]:
    m.pretty_print()

================================ Human Message =================================

Hi!, Search for the weather in Mumbai as of now
================================== Ai Message ==================================
Tool Calls:
  tavily_search (948edc51-6394-401d-a91c-ac2b67c37294)
 Call ID: 948edc51-6394-401d-a91c-ac2b67c37294
  Args:
    query: weather in Mumbai
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Mumbai", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Mumbai", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Mumbai', 'region': 'Maharashtra', 'country': 'India', 'lat': 18.975, 'lon': 72.826, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1756599229, 'localtime': '2025-08-31 05:43'}, 'current': {'last_updated_epoch': 1756598400, 'last_updated': '2025-08-31 05:30', 'temp_c': 27.1, 'temp_f': 80.8, 'is_day': 0, 'condition': {'text': 'Mist',

In [28]:
for inp in [resp1, resp2, resp3]:
    # Find the last HumanMessage in the turn
    last_human = next(m for m in reversed(inp["messages"]) if m.type == "human")
    msg_dict = {"role": "user", "content": last_human.content}

    for step in agent_executor.stream({"messages": [msg_dict]}, config=config, stream_mode="values"):
        step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi!, Search for the weather in Mumbai as of now
================================== Ai Message ==================================
Tool Calls:
  tavily_search (8f37c84c-f541-48c4-9962-84502b6d78e0)
 Call ID: 8f37c84c-f541-48c4-9962-84502b6d78e0
  Args:
    query: weather in Mumbai
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Mumbai", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Mumbai", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Mumbai', 'region': 'Maharashtra', 'country': 'India', 'lat': 18.975, 'lon': 72.826, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1756599229, 'localtime': '2025-08-31 05:43'}, 'current': {'last_updated_epoch': 1756598400, 'last_updated': '2025-08-31 05:30', 'temp_c': 27.1, 'temp_f': 80.8, 'is_day': 0, 'condition': {'text': 'Mist',

In [34]:
for inp in [resp1, resp2, resp3]:
    last_human = next(m for m in reversed(inp["messages"]) if m.type == "human")
    msg_dict = {"role": "user", "content": last_human.content}
    for step, metadata in agent_executor.stream({"messages": [msg_dict]}, config=config, stream_mode="messages"):
        if metadata["langgraph_node"] == "agent" and (text := step.text()):
            print(text, end="|")

The current weather in| Mumbai is Mist with a temperature of 27.1°C. The| wind is blowing from the SSW at 8.5 mph, the humidity| is 84%, and visibility is 3.0 km. The forecast for today is Mist with a maximum temperature of 29°C and a minimum of |25°C.|Given the current| misty conditions and a small amount of precipitation (0.27mm), it would| be wise to take a raincoat or umbrella with you in Mumbai.
|You first asked me| to search for the weather in Mumbai as of now.
|

In [33]:
for resp in [resp1, resp2, resp3]:
    last_ai = next(m for m in reversed(resp["messages"]) if m.type == "ai")
    print(last_ai.content)


The weather in Mumbai is Mist with a temperature of 27.1 degrees Celsius. The wind speed is 8.5 mph, humidity is 84%, and visibility is 3.0 km. The maximum temperature for today is expected to be 29°C and a minimum of 25°C.
Given the current weather condition of Mist and a precipitation of 0.27mm, it would be advisable to take a raincoat or an umbrella with you in Mumbai.
You first asked me to search for the weather in Mumbai as of now.
